In [1]:
import pandas as pd
import random

In [2]:
PATH_TRAIN = "C:/Users/kelis/ITU/Year_2/4th_semester/NLP/project/scripts/NLP_project2024-main/train.bio"
PATH_DEV = "C:/Users/kelis/ITU/Year_2/4th_semester/NLP/project/scripts/NLP_project2024-main/dev.bio"
PATH_TEST = "C:/Users/kelis/ITU/Year_2/4th_semester/NLP/project/scripts/NLP_project2024-main/test.bio"
label_all_tokens = True

In [4]:
def read_bio_file(path):
    
    data = []
    current_words = []
    current_tags = []

    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()[2:]
        
    for line in lines:
        
        line = line.strip()
        
        if line: # if line is not an empty line
            tok = line.split('\t')
            current_words.append(tok[0])
            current_tags.append(tok[3])
            
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []
            
            
    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

train_data = read_bio_file(PATH_TRAIN)
total_num_of_sentences = train_data.shape[0]

In [5]:
def deletion_sentence(sent, perc_words):
    
    n_words = int(len(sent) * perc_words)
    if n_words == 0:
        return sent
    
    word_idxs = [x for x in random.sample(list(range(len(sent))), n_words)]

    for i in range(n_words):
        
        type_of_mistake = random.randint(1,3)
        
        word = sent[word_idxs[i]]
        if len(word) > 1:
            # mistake type 1: missed last letter
            if type_of_mistake == 1:
                word = word[:-1]
            # mistake type 2: missed first letter
            elif type_of_mistake == 2:
                word = word[1:]
            # mistake type 3: missed random middle letter
            elif type_of_mistake == 3:
                if len(word) >= 3:
                    # make it so that you can't remove first or last letter
                    # and have to remove smth in the middle
                    del_idx = random.randint(1, len(word) - 2)
                    word = word[:del_idx] + word[del_idx+1:]
                
        sent[word_idxs[i]] = word
        
    return sent

In [22]:
s = ["This", "is", "a", "sentence", "test"]
new_s = deletion_sentence(s, 0.5)
print(new_s)

['This', 'is', 'a', 'sentenc', 'test']


In [23]:
def deletion_dataset(data, perc_sent, perc_words, prints=False):

    n_sent = int(perc_sent * data.shape[0])
    new_data = data.copy()
    
    idxs = [x for x in random.sample(list(range(data.shape[0])), n_sent)]
    
    for idx in idxs:
        
        if prints:
            print(f"Perturbing sentence idx {idx} | Original: ")
            print(data["words"][idx])
            
        new_sent = (deletion_sentence((new_data["words"][idx]).copy(), perc_words)).copy()
        new_data["words"][idx] = new_sent
                    
        if prints:
            print(f"Perturbed version:")
            print(new_data["words"][idx])
        
            print(data["words"][idx] == new_data["words"][idx])
        
    return len(idxs), idxs, new_data

In [24]:
p_sents = 0.2
p_words = 0.4

n_modified, idxs, perturbed_data = deletion_dataset(train_data, p_sents, p_words)
print(n_modified)
print(idxs)

327
[151, 20, 1252, 790, 1615, 82, 481, 341, 56, 395, 893, 1515, 119, 1553, 1566, 979, 1289, 285, 697, 1163, 1217, 1250, 1132, 610, 126, 138, 1485, 912, 990, 860, 830, 1086, 691, 1534, 1450, 1182, 242, 571, 368, 997, 149, 1416, 1610, 925, 72, 526, 340, 1038, 411, 625, 1276, 415, 58, 920, 275, 1316, 470, 446, 1317, 1189, 1027, 1274, 202, 463, 595, 769, 5, 870, 381, 837, 1122, 833, 261, 509, 109, 532, 342, 73, 118, 513, 90, 314, 420, 708, 974, 1636, 1616, 845, 558, 1545, 98, 572, 1582, 987, 322, 957, 618, 1107, 1224, 546, 1469, 1108, 701, 14, 1547, 1634, 65, 372, 1039, 1628, 989, 1093, 1187, 421, 763, 1368, 487, 695, 1513, 1141, 1271, 978, 206, 444, 749, 1239, 175, 689, 534, 3, 1019, 547, 654, 735, 945, 825, 1018, 120, 577, 471, 1098, 274, 904, 1575, 64, 993, 583, 884, 1195, 1350, 898, 458, 1296, 606, 142, 1155, 1002, 859, 719, 1116, 457, 1517, 445, 461, 995, 1421, 1046, 402, 1206, 451, 308, 1215, 617, 486, 1559, 907, 437, 976, 1500, 125, 404, 1397, 1478, 992, 542, 419, 1482, 1175, 1403,

C:\Users\kelis\AppData\Local\Temp\ipykernel_13108\3923332049.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["words"][idx] = new_sent
